In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# grofers 
def get_data_grofers(ptype, pageNo):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64;     x64; rv:66.0) Gecko/20100101 Firefox/66.0", \
               "Accept-Encoding":"gzip, deflate",     "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    
    #print("https://grofers.com/s/?q="+ptype)
    r = requests.get("https://grofers.com/s/?q="+ptype, headers=headers)
    
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup.prettify())
    
    #x=soup.find('div', attrs={'class':'plp-product'})
    #print(x)
    
    p_product_df = pd.DataFrame(columns=['Category','Product Name','UoM','MRP','Selling Price', 'Club Price'])
    
    for d in soup.findAll('div', attrs={'class':'plp-product'}):
        name = d.find('div', attrs={'class':'plp-product__name--box'})
        sellingprice = d.find('span', attrs={'class':'plp-product__price--new'})
        mrp = d.find('div', attrs={'class':'plp-product__price--old display--inline-block@mobile'})
        club = d.find('div', attrs={'class':'plp-product__non-member-price'})
        qty = d.find('div', attrs={'class':'plp-product__quantity'})
    
        #print(rating.text)   
        if name is not None:
            final_name = name.text
        else:
            final_name = "unknown-product"
       
        if sellingprice is not None:
            last_sellingprice = sellingprice.text
            last_sellingprice = last_sellingprice[1:]
        else:
            last_sellingprice = '0'
        
        if mrp is not None:
            mrp_price = mrp.text
            mrp_price = mrp_price[1:]
        else:
            mrp_price = last_sellingprice

        if club is not None:
            clubprice = club.text
            clubprice = clubprice[1:]
        else:
            clubprice = last_sellingprice  

        if qty is not None:
            final_qty = qty.text
        else:
            final_qty = '-1'
        
        #print(ptype,final_name, last_sellingprice, final_qty)
        #product_df.append({'Product Name': 'acer','Selling Price':'1000$','Rating':'4 out of 5'}, ignore_index=True)
        p_product_df = p_product_df.append({'Category':ptype,'Product Name': final_name,'MRP':mrp_price,'Selling Price':last_sellingprice,'Club Price':clubprice,'UoM':final_qty}, ignore_index=True)
        
    return p_product_df

In [3]:
data_df = pd.DataFrame(columns=['Category','Product Name','UoM','MRP','Selling Price', 'Club Price'])
no_pages = 1

#ptype_list = ['chocolates']
ptype_list = ['namkeen-snacks']

for x in ptype_list:
    for i in range(no_pages):
        data_df = data_df.append(get_data_grofers(x,i),ignore_index=True)

In [4]:
data_df.to_csv('3.csv')
data_df.head()

,Category,Product Name,UoM,MRP,Selling Price,Club Price
0,namkeen-snacks,Haldiram's Bhujia Namkeen,1 kg,225,188,lub Price: ₹182
1,namkeen-snacks,Grofers Happy Day Bhujia Sev Namkeen,1 kg,215,148,lub Price: ₹143
2,namkeen-snacks,Bikaneri Bhujia Namkeen,950 g,151,125,lub Price: ₹121
3,namkeen-snacks,Haldiram's Boondi Namkeen,200 g,44,44,lub Price: ₹42
4,namkeen-snacks,Grofers Happy Day Soya Katori Namkeen,100 g,45,27,lub Price: ₹26
